<a href="https://colab.research.google.com/github/MashaKubyshina/Learning_to_code/blob/master/Classification_Model_Training_Job_Postings_(PART_2_FEATURE_ENGINEERING).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [271]:
# The Feature Engineering is done following this path https://towardsdatascience.com/text-classification-in-python-dd95d264c802
# To install it, please type this command in the shell(terminal): ! conda install -c conda-forge altair vega_datasets notebook vega

**Feature Engineering**

In this part we will transform the data in a set of inputs for our model
Having good quality features helps us improve the model

Dealing with text data there are several way to obtain features to represent the data



1.   Text representation
2.   List item





**Text Cleaning and Preparation**

There are varios feature creation methods
These methods treat each row in the dataset as a separate document. These both methods are often named "Bag of Words" methods. They don't take into account the order of words.

1.   Count word vectors
2.   TF–IDF Vectors


"TF-IDF is a score that represents the relative importance of a term in the document and the entire corpus. TF stands for Term Frequency, and IDF stands for Inverse Document Frequency." from https://towardsdatascience.com/text-classification-in-python-dd95d264c802

Other, more sophisticated methods are:


1.   Word Embeddings
2.   Text based or NLP based features




In [272]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

In [273]:
# Open the dataset from https://data.world/jobspikr/10000-data-scientist-job-postings-from-the-usa
# This dataset has 10000 entries of job postings in data science; the two columns we are particulary interested in are:
# category and job_description

import pandas as pd
df = pd.read_csv('https://query.data.world/s/hpv4ejstsxjszdo2eathrfgjhzqlqy')
df.head(10)

,crawl_timestamp,url,job_title,category,company_name,city,state,country,inferred_city,inferred_state,inferred_country,post_date,job_description,job_type,salary_offered,job_board,geo,cursor,contact_email,contact_phone_number,uniq_id,html_job_description
0,2019-02-06 05:26:22 +0000,https://www.indeed.com/viewjob?jk=fd83355c2b23...,Enterprise Data Scientist I,Accounting/Finance,Farmers Insurance Group,Woodland Hills,CA,Usa,Woodland hills,California,Usa,2019-02-06,Read what people are saying about working here...,Undefined,NaN,indeed,usa,1549432819114777,NaN,NaN,3b6c6acfcba6135a31c83bd7ea493b18,NaN
1,2019-02-06 05:33:41 +0000,https://www.dice.com/jobs/detail/Data-Scientis...,Data Scientist,NaN,Luxoft USA Inc,Middletown,NJ,Usa,Middletown,New jersey,Usa,2019-02-05,We have an immediate opening for a Sharp Data ...,Undefined,NaN,dice,usa,1549432819122106,NaN,NaN,741727428839ae7ada852eebef29b0fe,NaN
2,2019-02-06 05:33:35 +0000,https://www.dice.com/jobs/detail/Data-Scientis...,Data Scientist,NaN,Cincinnati Bell Technology Solutions,New York,NY,Usa,New york,New york,Usa,2019-02-05,Candidates should have the following backgroun...,Full Time,NaN,dice,usa,1549432819236156,NaN,NaN,cdc9ef9a1de327ccdc19cc0d07dbbb37,NaN
3,2019-02-06 05:33:42 +0000,https://www.indeed.com/viewjob?jk=841edd86ead2...,"Data Scientist, Aladdin Wealth Tech, Associate...",Accounting/Finance,BlackRock,New York,NY 10055 (Midtown area),Usa,New york,New york,Usa,2019-02-06,Read what people are saying about working here...,Undefined,NaN,indeed,usa,1549432819259473,NaN,NaN,1c8541cd2c2c924f9391c7d3f526f64e,NaN
4,2019-02-06 05:48:23 +0000,https://job-openings.monster.com/senior-data-s...,Senior Data Scientist,biotech,CyberCoders,Charlotte,NC,Usa,Charlotte,North carolina,Usa,2019-02-05,We are seeking an extraordinary Data Scientist...,Full Time,NaN,monster,usa,1549436429015957,NaN,NaN,445652a560a5441060857853cf267470,NaN
5,2019-02-06 05:36:06 +0000,https://www.indeed.com/viewjob?jk=14fa3772cd5f...,CIB – Fixed Income Research – Machine Learning...,Accounting/Finance,JP Morgan Chase,New York,NY 10179 (Midtown area),Usa,New york,New york,Usa,2019-02-05,Read what people are saying about working here...,Undefined,NaN,indeed,usa,1549436429033307,NaN,NaN,9571ec617ba209fd9a4f842973a4e9c8,NaN
6,2019-02-06 05:34:37 +0000,https://www.indeed.com/viewjob?jk=c6db96b37f8a...,"Data Scientist, Licensing Operations",Accounting/Finance,Spotify,New York,NY 10011 (Chelsea area),Usa,New york,New york,Usa,2019-02-06,Read what people are saying about working here...,Undefined,NaN,indeed,usa,1549436429042348,NaN,NaN,0ec629c03f3e82651711f2626c23cadb,NaN
7,2019-02-06 05:52:12 +0000,https://www.dice.com/jobs/detail/Sr.-Data-Scie...,Sr. Data Scientist (Can work on Xoriant W2),NaN,Xoriant Corporation,Santa Clara,CA,Usa,Santa clara,California,Usa,2019-02-06,Job Title: - Sr. Data Science Consultant Durat...,Contract,NaN,dice,usa,1549436429042523,NaN,NaN,972e897473d65f34b8e7f1c1b4c74b1c,NaN
8,2019-02-06 05:34:18 +0000,https://www.indeed.com/viewjob?jk=0fc298b9f3a8...,"Data Scientist, Aladdin Wealth Tech, Associate",Accounting/Finance,BlackRock,New York,NY 10055 (Midtown area),Usa,New york,New york,Usa,2019-02-06,Read what people are saying about working here...,Undefined,NaN,indeed,usa,1549436429066810,NaN,NaN,80d64b46bc7c89602f63daf06b9f1b4c,NaN
9,2019-02-06 06:03:55 +0000,https://www.dice.com/jobs/detail/Data-Scientis...,Data Scientist,NaN,Adroit Resources,San Francisco,CA,Usa,San francisco,California,Usa,2019-02-05,• 3+ years related a professional experience ...,Contract,NaN,dice,usa,1549436429884667,NaN,NaN,b772c6ef8ee7631895ab9a59b5e8b2c1,NaN


In [274]:
# Visualize 1 sample job description

df.loc[1]['job_description']

'We have an immediate opening for a Sharp Data Scientist with a strong Mathematical/Statistical background to work on various initiatives in a Huge data environment. Will be looking at various formats of Data dealing with Billing Fraud Patterns using various software and modeling techniques. The candidate should have a minimum of 3 years of solid work experience in a professional organization/corporation using Python, SQL, Python libraries, Machine Learning Algorithms ( i.e. Decision Tree, Random Forest, Logistic Regression, etc.) should have strong Python skills. Additional experience with R Cloud would be a plus. Will be working with structured and unstructured data formats so any experience with Hive, and other Big Data technologies would be a plus. Should have strong communication skills and the ability to explain data findings to both Technical and Non- Technical audiences. We are NOT considering Recent College Grads- MUST have a minimum of 2-3 post Graduate experience working in 

In [275]:
# special characters cleaning
# \r and \n

df['Content_Parsed_1'] = df['job_description'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")

In [276]:
# " when quoting text

df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')

In [277]:
# Lowercasing the text

df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()

In [278]:
#punctuation doesn't have any prediction, thus we will clean it

punctuation_signs = list("?:!.,;")
df['Content_Parsed_3'] = df['Content_Parsed_2']

for punct_sign in punctuation_signs:
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')

In [279]:
# removing possesive pronouns termination

df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")

In [280]:
# Stemming and Lemmatization
# Since stemming can produce output words that don't exist, we'll only use a lemmatization process at this moment. 
# Lemmatization takes into consideration the morphological analysis of the words and returns words that do exist, 
# so it will be more useful for us.

# Downloading punkt and wordnet from NLTK
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
------------------------------------------------------------
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [281]:
# Saving the lemmatizer into an object

wordnet_lemmatizer = WordNetLemmatizer()

In [282]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

In [283]:
df['Content_Parsed_5'] = lemmatized_text_list

In [284]:
# Downloading the stop words list

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [285]:
# Loading the stop words in english

stop_words = list(stopwords.words('english'))

In [286]:
# checking stopwords

stop_words[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [287]:
# we will use regular expressions to delete whole words

df['Content_Parsed_6'] = df['Content_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')

In [288]:
# let's check the difference between original text and parsed text
# original text

df.loc[5]['job_description']

'Read what people are saying about working here. \n\nOpportunity\n\nThe opportunity is to join our New York team as an analyst, associate, or VP, with a focus on applications of machine learning and artificial intelligence in fixed income/securitized products markets. The role requires an individual with excellent technical and programming skills, preferably with an advanced degree in quantitative field (e.g., physics, computer science, mathematics, engineering).\n\nOur business\n\nJ.P. Morgan has the leading Global Rates and Global Spread business in terms of volume traded, issuers traded and quality of investor relationships. The Rates business covers Treasuries, swaps, options and other derivatives, while Spreads covers Credit, SPG, and Public Finance Markets. J.P. Morgan Global Spread and Trading offers first-class, highly integrated financial services to a global client base and provides financial assets and liquidity for banks, insurance companies, finance companies, mutual funds

In [289]:
# parsed text

df.loc[5]['Content_Parsed_6']

'read  people  say  work    opportunity   opportunity   join  new york team   analyst associate  vp   focus  applications  machine learn  artificial intelligence  fix income/securitized products market  role require  individual  excellent technical  program skills preferably   advance degree  quantitative field (eg physics computer science mathematics engineering)   business  jp morgan   lead global rat  global spread business  term  volume trade issuers trade  quality  investor relationships  rat business cover treasuries swap options   derivatives  spread cover credit spg  public finance market jp morgan global spread  trade offer first-class highly integrate financial service   global client base  provide financial assets  liquidity  bank insurance company finance company mutual fund  hedge fund traders salespeople  research analysts work collectively  generate ideas  credit business make secondary market  high grade bonds/cds high yield bonds/cds distress bond indices options corre

In [290]:
df.head(1)

,crawl_timestamp,url,job_title,category,company_name,city,state,country,inferred_city,inferred_state,inferred_country,post_date,job_description,job_type,salary_offered,job_board,geo,cursor,contact_email,contact_phone_number,uniq_id,html_job_description,Content_Parsed_1,Content_Parsed_2,Content_Parsed_3,Content_Parsed_4,Content_Parsed_5,Content_Parsed_6
0,2019-02-06 05:26:22 +0000,https://www.indeed.com/viewjob?jk=fd83355c2b23...,Enterprise Data Scientist I,Accounting/Finance,Farmers Insurance Group,Woodland Hills,CA,Usa,Woodland hills,California,Usa,2019-02-06,Read what people are saying about working here...,Undefined,NaN,indeed,usa,1549432819114777,NaN,NaN,3b6c6acfcba6135a31c83bd7ea493b18,NaN,Read what people are saying about working here...,read what people are saying about working here...,read what people are saying about working here...,read what people are saying about working here...,read what people be say about work here we b...,read people say work farmers join te...


In [291]:
# we will rearrange the dataset and only use the columns we need

list_columns = ['uniq_id','job_title', 'category', 'company_name', 'state', 'post_date','job_type', 'job_description', 'Content_Parsed_6']
df = df[list_columns]

df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})

In [292]:
df.head()

,uniq_id,job_title,category,company_name,state,post_date,job_type,job_description,Content_Parsed
0,3b6c6acfcba6135a31c83bd7ea493b18,Enterprise Data Scientist I,Accounting/Finance,Farmers Insurance Group,CA,2019-02-06,Undefined,Read what people are saying about working here...,read people say work farmers join te...
1,741727428839ae7ada852eebef29b0fe,Data Scientist,NaN,Luxoft USA Inc,NJ,2019-02-05,Undefined,We have an immediate opening for a Sharp Data ...,immediate open sharp data scientist str...
2,cdc9ef9a1de327ccdc19cc0d07dbbb37,Data Scientist,NaN,Cincinnati Bell Technology Solutions,NY,2019-02-05,Full Time,Candidates should have the following backgroun...,candidates follow background skills charac...
3,1c8541cd2c2c924f9391c7d3f526f64e,"Data Scientist, Aladdin Wealth Tech, Associate...",Accounting/Finance,BlackRock,NY 10055 (Midtown area),2019-02-06,Undefined,Read what people are saying about working here...,read people say work blackrock blackro...
4,445652a560a5441060857853cf267470,Senior Data Scientist,biotech,CyberCoders,NC,2019-02-05,Full Time,We are seeking an extraordinary Data Scientist...,seek extraordinary data scientist charlott...


**Subsetting the dataset**

In [293]:
# we will create a dictionary with the labels for categories (this is where our previous work of categorizing will come handy)

# For this project I will work with the data from 3 categories that are balanced, I will repeat her ethe code from EDA-Part 1


df=df.dropna(how='any',axis=0)

df.shape

(8702, 9)

In [294]:
# splitting the long names into words

df['category']=df['category'].str.split().str[0]
df['category'].unique()

array(['Accounting/Finance', 'biotech', 'Computer/Internet',
       'Arts/Entertainment/Publishing', 'military', 'business',
       'Engineering/Architecture', 'Manufacturing/Mechanical', 'life',
       'Banking/Loans', 'agriculture', 'Education/Training', 'science',
       'arts', 'Customer+Service', 'food', 'technology', 'Healthcare',
       'Government/Military', 'architecture', 'community', 'production',
       'r', 'Insurance', 'security', 'healthcare', 'Human+Resources',
       'Administrative', 'language', 'media', 'Sales', 'advertising',
       'office', 'Restaurant/Food+Service', 'Telecommunications',
       'Construction/Facilities', 'Legal', 'Marketing/Advertising/PR',
       'Non-Profit/Volunteering', 'Pharmaceutical/Bio-tech', 'education',
       'management', 'transportation', 'protective', 'legal', 'sales',
       'Retail', 'Hospitality/Travel', 'building',
       'Transportation/Logistics', 'math', 'Upper+Management/Consulting',
       'Accounting-or-finance', 'Computer

In [295]:
# I will solidify categories

df['category']= df['category'].replace(['Accounting/Finance','Accounting-or-finance','finance','financieel','Banking','Banking/Loans','Banking-or-loans'],'Finance')
df['category']= df['category'].replace(['biotech','Pharmaceutical/Bio-tech','science','math'],'Biotech')
df['category']= df['category'].replace(['Computer/Internet','technology','Computer-or-internet','computer'],'Computer/Tech')
df['category']= df['category'].replace(['life','social','r','animal','higher','personal'],'Misc')
df['category']= df['category'].replace(['Arts/Entertainment/Publishing','Arts-or-entertainment-or-publishing','entertainment','Entertainment','production','media','community','advertising','Marketing/Advertising/PR','Sales','sales','Marketing-or-advertising-or-pr','SocialMedia/Marketing'],'Marketing/Ads/Entertainment')
df['category']= df['category'].replace(['military','Government/Military','science','Government-or-military'],'Military')
df['category']= df['category'].replace(['business','management','Upper-Management-or-consulting'],'Business/Consulting')
df['category']= df['category'].replace(['Administrative','office','administrative','Human+Resources','Human-Resources','Customer+Service','Telecommunications','service','Customer-Service','CustomerService'],'Administrative/HR')
df['category']= df['category'].replace(['Engineering/Architecture','architecture','Engineering-or-architecture','engineering'],'Engineering/Architecture')
df['category']= df['category'].replace(['Manufacturing/Mechanical','production','manufacturing','Manufacturing-or-mechanical'],'Manufacturing')
df['category']= df['category'].replace(['agriculture',],'Agriculture')
df['category']= df['category'].replace(['Restaurant/Food+Service','food','Restaurant-or-food-Service'],'FoodService')
df['category']= df['category'].replace(['Education/Training','education','Education-or-training'],'Education/Training')
df['category']= df['category'].replace(['science','math'],'Math/Science')
df['category']= df['category'].replace(['Healthcare','healthcare'],'Healthcare')
df['category']= df['category'].replace(['Insurance','protective'],'Insurance')
df['category']= df['category'].replace(['security','Law-Enforcement-or-security'],'Security')
df['category']= df['category'].replace(['language','arts','Arts'],'Language/Arts')
df['category']= df['category'].replace(['Construction/Facilities','building','Construction-or-facilities','construction'],'Construction')
df['category']= df['category'].replace(['Legal','legal'],'Legal')
df['category']= df['category'].replace(['transportation','Transportation/Logistics','Transportation-or-logistics'],'Transportation')
df['category']= df['category'].replace(['Hospitality-or-travel','Hospitality/Travel'],'Hospitality/Travel')

In [296]:
df['category'].unique()

array(['Finance', 'Biotech', 'Computer/Tech',
       'Marketing/Ads/Entertainment', 'Military', 'Business/Consulting',
       'Engineering/Architecture', 'Manufacturing', 'Misc', 'Agriculture',
       'Education/Training', 'Language/Arts', 'Administrative/HR',
       'FoodService', 'Healthcare', 'Insurance', 'Security',
       'Construction', 'Legal', 'Non-Profit/Volunteering',
       'Transportation', 'Retail', 'Hospitality/Travel',
       'Upper+Management/Consulting', 'Non-profit/volunteering',
       'banking', 'Pharmaceutical/bio-tech', 'Science', 'communications',
       'Admin', 'executive', 'Data', 'Senior', 'TV', 'Principal', 'Vice',
       'Sr.', 'Applied', 'Digital', 'DISTINGUISHED', 'Manager,', 'Lead',
       'Quality', 'Mid', 'Sr', 'Agile', 'Intel', 'Staff', 'Clinical',
       'Customer', 'Systems,', 'Geospatial', 'Mgr/Sr', 'Deep',
       'Marketing', 'Optimization', 'AI', 'Research', 'Revenue',
       'Associate', 'Full-time', 'Actuarial', 'Homeland', 'Distinguished',
   

In [297]:
# I will use only 3 categories here

df=df[(df.category == 'Computer/Tech') | (df.category == 'Engineering/Architecture') | (df.category == 'Business/Consulting')]
df.head(10)

,uniq_id,job_title,category,company_name,state,post_date,job_type,job_description,Content_Parsed
10,9cd3ed78e5cac9e516ea41173de2c25f,Data Scientist,Computer/Tech,Northrop Grumman,CA 93940,2019-02-06,Undefined,Read what people are saying about working here...,read people say work northrop grumman i...
12,53224da901548e7137bbb163d456ba6a,ETL Developer / Data Scientist,Computer/Tech,Noblis,VA 20191,2019-02-06,Undefined,Read what people are saying about working here...,read people say work responsibilities ...
13,010b5d671896d26eba50948c7c337f94,Research Data Scientist,Computer/Tech,ARUP Laboratories,UT,2019-02-06,Undefined,Read what people are saying about working here...,read people say work job detail descrip...
18,4a8b875d1acc4f560716561d699aa022,Data Scientist,Computer/Tech,Bank of America,WA 98104 (First Hill area),2019-02-06,Undefined,Read what people are saying about working here...,read people say work job description po...
19,edcdf4dd38cca4bbccd5ba0b787d2c49,Data Scientist,Computer/Tech,Sallie Mae,DE,2019-02-06,Undefined,Read what people are saying about working here...,read people say work operations strateg...
21,2956d0023b0c4617c430adcedb1e38d9,Data Scientist,Computer/Tech,VideoAmp,CA,2019-02-06,Undefined,Read what people are saying about working here...,read people say work us videoamp’ miss...
22,e457f78180e72dd1ddee0efc042b0496,Data Scientist,Computer/Tech,Apple,MA,2019-02-06,Undefined,Read what people are saying about working here...,read people say work summary post feb 5...
32,b6e361550638f071a4985bf5f3d440ce,Data Scientist,Business/Consulting,Apex Systems,OH,2019-02-05,Full Time,Description Building large systematic reports ...,description build large systematic report one...
33,d33577ea9ae09c58d77e1fab2c012ba2,"Senior Data Scientist - Tallahassee, FL - $150...",Business/Consulting,Jefferson Frank,FL,2019-02-05,Full Time,My client is a leader in the Manufacturing ver...,client leader manufacture vertical oper...
34,09378ddde9b997b1acbf519c2b9ddf03,Data Scientist,Business/Consulting,"5 Star Global Recruitment Partners, LLC",TX,2019-02-05,Full Time,"Data Scientist Client: A Fortune 500, very lar...",data scientist client fortune 500 large mult...


In [298]:
df.shape

(4664, 9)

In [299]:
df=df.iloc[:2000]

In [300]:
df.shape

(2000, 9)

In [301]:
df.describe

<bound method NDFrame.describe of                                uniq_id  ...                                     Content_Parsed
10    9cd3ed78e5cac9e516ea41173de2c25f  ...  read  people  say  work     northrop grumman i...
12    53224da901548e7137bbb163d456ba6a  ...  read  people  say  work    responsibilities   ...
13    010b5d671896d26eba50948c7c337f94  ...  read  people  say  work    job detail  descrip...
18    4a8b875d1acc4f560716561d699aa022  ...  read  people  say  work    job description  po...
19    edcdf4dd38cca4bbccd5ba0b787d2c49  ...  read  people  say  work     operations strateg...
...                                ...  ...                                                ...
4216  fae59097315afaf0dfc4c49327c29f55  ...   good people build reward career   think  work...
4217  d54a1ab24d42b54b9d95f6d3116accac  ...  senior data scientist machine learn   thrive k...
4218  2cfcbe7a850a5b9dabc04622602faa4f  ...   good people build reward career   think  work...
4219  899bccb75f

**Label Coding**

In [302]:
category_codes = {
    'Computer/Tech': 0,
    'Engineering/Architecture': 1,
    'Business/Consulting': 2
}

In [303]:
# Category mapping

df['Category_Code'] = df['category']
df = df.replace({'Category_Code':category_codes})

In [304]:
df.head()

,uniq_id,job_title,category,company_name,state,post_date,job_type,job_description,Content_Parsed,Category_Code
10,9cd3ed78e5cac9e516ea41173de2c25f,Data Scientist,Computer/Tech,Northrop Grumman,CA 93940,2019-02-06,Undefined,Read what people are saying about working here...,read people say work northrop grumman i...,0
12,53224da901548e7137bbb163d456ba6a,ETL Developer / Data Scientist,Computer/Tech,Noblis,VA 20191,2019-02-06,Undefined,Read what people are saying about working here...,read people say work responsibilities ...,0
13,010b5d671896d26eba50948c7c337f94,Research Data Scientist,Computer/Tech,ARUP Laboratories,UT,2019-02-06,Undefined,Read what people are saying about working here...,read people say work job detail descrip...,0
18,4a8b875d1acc4f560716561d699aa022,Data Scientist,Computer/Tech,Bank of America,WA 98104 (First Hill area),2019-02-06,Undefined,Read what people are saying about working here...,read people say work job description po...,0
19,edcdf4dd38cca4bbccd5ba0b787d2c49,Data Scientist,Computer/Tech,Sallie Mae,DE,2019-02-06,Undefined,Read what people are saying about working here...,read people say work operations strateg...,0


**Train and Test Splits**

In [305]:
# Since we have a large number of entries, we are good using 25% for testing

X_train, X_test, y_train, y_test = train_test_split(df['Content_Parsed'], 
                                                    df['Category_Code'], 
                                                    test_size=0.25, 
                                                    random_state=8)

**Text Representation**

We will use TF-IDF Vectors as features in this project.

We have to define the different parameters:
*  ngram_range: We want to consider both unigrams and bigrams.
*  max_df: When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold
*  min_df: When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold.
*  max_features: If not None, build a vocabulary that only consider the top *  max_features ordered by term frequency across the corpus.


For more information refer to this tutorial https://github.com/miguelfzafra/Latest-News-Classifier/blob/master/0.%20Latest%20News%20Classifier/03.%20Feature%20Engineering/03.%20Feature%20Engineering.ipynb


In [306]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

In [307]:
# We can try different combination of the params to improve accuracy of the models

tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print("Train features", features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print("Test features", features_test.shape)

# we have fitted and then transformed the training set, but we have only transformed the test set.
# for train we have: tfidf.fit_transform and for test: tfidf.transform

Train features (1500, 300)
Test features (500, 300)


In [308]:
# We can use the Chi squared test in order to see what unigrams and bigrams are most correlated with each category:

from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(category_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-3:])))
    print("")

# 'Business/Consulting' category:
  . Most correlated unigrams:
. read
. please
. say
. aws
. client
  . Most correlated bigrams:
. read people
. say work
. people say

# 'Computer/Tech' category:
  . Most correlated unigrams:
. code
. computer
. please
. client
. aws
  . Most correlated bigrams:
. say work
. computer science
. deep learn

# 'Engineering/Architecture' category:
  . Most correlated unigrams:
. client
. mission
. write
. power
. security
  . Most correlated bigrams:
. say work
. read people
. people say



In [309]:
# we can see that bigrams and unigrams correspond well to the topic; bigrams correspond better
# we can check the birgams

[bigrams]

[['sexual orientation',
  'national origin',
  'machine learn',
  'veteran status',
  'predictive model',
  'data set',
  'relate field',
  'gender identity',
  'data source',
  'data science',
  'experience work',
  'equal opportunity',
  'data mine',
  'communication skills',
  'data analysis',
  'years experience',
  'data analytics',
  'opportunity employer',
  'deep learn',
  'computer science',
  'data scientists',
  'experience data',
  'data scientist',
  'bachelor degree',
  'big data',
  'say work',
  'read people',
  'people say']]

In [310]:
# we will save the files we need for next steps
# for this code to work you need to first create directory
# I used google drive to be able to access these files elsewhere

# X_train
with open('/content/drive/MyDrive/Pickles/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open('/content/drive/MyDrive/Pickles/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open('/content/drive/MyDrive/Pickles/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open('/content/drive/MyDrive/Pickles/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open('/content/drive/MyDrive/Pickles/df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
# features_train
with open('/content/drive/MyDrive/Pickles/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open('/content/drive/MyDrive/Pickles/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open('/content/drive/MyDrive/Pickles/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open('/content/drive/MyDrive/Pickles/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open('/content/drive/MyDrive/Pickles/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)